In [71]:
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import glob 
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math

In [72]:
user=pd.read_csv(r"C:\Users\Silmi\Desktop\relax_challenge\takehome_users.csv",encoding='latin-1')

In [73]:
user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [74]:
user_engagement=pd.read_csv(r"C:\Users\Silmi\Desktop\relax_challenge\takehome_user_engagement.csv",encoding='latin-1')

In [75]:
user_engagement.shape,user.shape

((207917, 3), (12000, 10))

Users who has logged into the product on three separate
days in at least one seven-day period are considered active.


In [76]:
user_engagement.head(2)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1


In [77]:
user_engagement['time_stamp_str'] =user_engagement['time_stamp'].astype(str).str[:10]

In [78]:
user_engagement['time_stamp'] =pd.to_datetime(user_engagement['time_stamp'])

In [79]:
user_engagement.head(2)

,time_stamp,user_id,visited,time_stamp_str
0,2014-04-22 03:53:30,1,1,2014-04-22
1,2013-11-15 03:45:04,2,1,2013-11-15


In [80]:
user_engagement_grouped=user_engagement.groupby(['user_id','time_stamp']).agg({'visited':np.sum})

In [81]:
user_engagement_grouped.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     207917 non-null  int64         
 1   time_stamp  207917 non-null  datetime64[ns]
 2   visited     207917 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [82]:
df=user_engagement_grouped.reset_index().groupby('user_id').resample('W',on='time_stamp').agg('sum')

In [83]:
df.rename(columns={'user_id':'junk_column'},inplace=True)

In [84]:
df=df.reset_index().drop('junk_column',axis=1)

In [85]:
active_users=df[df['visited']>=3]['user_id'].values

In [86]:
len(active_users)

33859

In [87]:
active_list = list(set(active_users))

In [88]:
len(active_list)

1445

In [89]:
user['active'] = np.where(user['object_id'].isin(active_list) , 0, 1)

In [90]:
user['active'].value_counts()

1    10555
0     1445
Name: active, dtype: int64

In [91]:
user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1


Identify which factors predict future user adoption

In [93]:
user['invited'] = np.where(user['invited_by_user_id'].isnull() , 1, 0)

In [94]:
user['creation_source'].value_counts()

ORG_INVITE            4254
GUEST_INVITE          2163
PERSONAL_PROJECTS     2111
SIGNUP                2087
SIGNUP_GOOGLE_AUTH    1385
Name: creation_source, dtype: int64

In [95]:
user_df=user[['creation_source','opted_in_to_mailing_list','enabled_for_marketing_drip','invited','active']]

In [96]:
user_df_dummies=pd.get_dummies(user_df)

In [97]:
X=user_df_dummies.drop('active',axis=1)
y=user_df_dummies.active.values

In [98]:
clf=AdaBoostClassifier()

In [99]:
clf.fit(X,y)

AdaBoostClassifier()

In [100]:
clf.feature_importances_

array([0.22, 0.16, 0.04, 0.08, 0.08, 0.08, 0.04, 0.16])

Note :opted in to mailing & enabled for marketing drip predict future user
adoption the best! Both are responsible for 38% variance.